In [1]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import Perceptron

In [2]:
comments = []
with open('TxtData/datacomment.txt','r', encoding = 'utf8') as f:
    comments = f.read().split('\n');

In [3]:
labels = []
with open('TxtData/datalabel.txt','r', encoding = 'utf8') as f:
    labels = f.read().split('\n');

In [4]:
comments[:5]

['dùng hơn tháng , ko nhận_diện đc khuông mặt , 1 số ứng_dụng tự_động dừng , chơi game máy lang',
 'camera ok lắm . mấy thì giá này nói_gì nữa . ai xài thấy sao chứ mình thấy thích nha . nhất là giao_diện của máy . mình là sam fan mà . sam là thích hết',
 'mình sài 2 sim mobi cùng lúc , nhưng lúc người khác gọi được cho mình lúc không , đã đổi máy vẫn vậy cho đến khi chỉ dùng 1 sim thì mới hết',
 'mình mua máy cách đây 6 tháng giờ gọi video zalo nói người khác ko nghe đc , đã khôi_phục cài_đặt mà vẫn ko đc , có gì hỗ_trợ mình',
 'thất_vọng về chất_lượng máy . lướt web chậm , giật lag . cảm_ứng chậm , lag . nói chug tầm giá này có_thể mua đc máy tốt hơn']

In [5]:
# ================== process raw data ===========
import re
import pyvi
from pyvi import ViTokenizer 

REPLACE_NO_SPACE = re.compile("[.;:\',\"()\[\]][.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    reviews = [line.lstrip() for line in reviews]
    reviews = [ViTokenizer.tokenize(rev) for rev in reviews]
    return reviews
comments = preprocess_reviews(comments)

In [6]:
comments[:5]

['dùng hơn tháng , ko nhận_diện đc khuông mặt , 1 số ứng_dụng tự_động dừng , chơi game máy lang',
 'camera ok lắm . mấy thì giá này nói_gì nữa . ai xài thấy sao chứ mình thấy thích nha . nhất là giao_diện của máy . mình là sam fan mà . sam là thích hết',
 'mình sài 2 sim mobi cùng lúc , nhưng lúc người khác gọi được cho mình lúc không , đã đổi máy vẫn vậy cho đến khi chỉ dùng 1 sim thì mới hết',
 'mình mua máy cách đây 6 tháng giờ gọi video zalo nói người khác ko nghe đc , đã khôi_phục cài_đặt mà vẫn ko đc , có gì hỗ_trợ mình',
 'thất_vọng về chất_lượng máy . lướt web chậm , giật lag . cảm_ứng chậm , lag . nói chug tầm giá này có_thể mua đc máy tốt hơn']

In [7]:
cv = CountVectorizer(binary=True)
cv.fit(comments)
commentsVec = cv.transform(comments)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    commentsVec, labels, train_size = 0.8
)

C:\Users\Thuy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
def SVM():
    clf = SVC(kernel = 'linear', C = 1e5) # just a big number 
    clf.fit(X_train.toarray(), y_train) 
    w = clf.coef_
    b = clf.intercept_
    #print('w = ', w)
    #print('b = ', b)
    print ("Final Accuracy SVM: %s"% accuracy_score(y_test, clf.predict(X_test.toarray())))

In [10]:
def multiNB():
    clf = MultinomialNB()
    clf.fit(X_train, y_train)

    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)

    acc_test = accuracy_score(y_pred_test,y_test)
    acc_train = accuracy_score(y_pred_train,y_train)
    #print('accuracy_train: ', acc_train)
    print('Final Accuracy multi Naive Bayes: ', acc_test)

In [11]:
def perceptron():
    clf = Perceptron(tol=1e-3, random_state=0)
    clf.fit(X_train, y_train)
    acc_train=clf.score(X_train, y_train)
    acc_test =clf.score(X_test, y_test)
    #print('accurracy_train of folder {}: ', acc_train)
    print('Final Accuracy perceptron: ', acc_test)

In [12]:
X_trainlog, X_testlog, y_trainlog, y_testlog = train_test_split(
    X_train, y_train, train_size = 0.75
)
def logistic():
    cmax=0.5
    accmax=0
    for c in [0.01, 0.05, 0.25, 0.5, 1]:
        lr = LogisticRegression(C=c)
        lr.fit(X_trainlog, y_trainlog)
        acc = accuracy_score(y_testlog, lr.predict(X_testlog))
        if(acc>accmax):
            accmax=acc
            cmax=c
    final_model = LogisticRegression(C=cmax)
    final_model.fit(X_train, y_train)
    print ("Final Accuracy logistic: %s"% accuracy_score(y_test, final_model.predict(X_test)))

In [13]:
multiNB()
logistic()
perceptron()
SVM()

Final Accuracy multi Naive Bayes:  0.8193069306930693


C:\Users\Thuy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy logistic: 0.8242574257425742
Final Accuracy perceptron:  0.754950495049505
Final Accuracy SVM: 0.7202970297029703


In [105]:
#raw noi tu ghep l2
multiNB()
logistic()
perceptron()
SVM()

Final Accuracy multi Naive Bayes:  0.8649350649350649


C:\Users\Thuy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy logistic: 0.8363636363636363
Final Accuracy perceptron:  0.8025974025974026
Final Accuracy SVM: 0.7714285714285715
